# Feature extraction

In [12]:
!pip install -U yt-dlp==2023.1.6 matplotlib==3.6.0 datasets[audio]

Cloning into 'sota-music-tagging-models'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 322
Receiving objects: 100% (325/325), 436.62 MiB | 16.73 MiB/s, done.
Resolving deltas: 100% (142/142), done.
Updating files: 100% (60/60), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import sys
sys.path.insert(1, './sota-music-tagging-models/training')

import model
from musiccaps import load_musiccaps

from tqdm.auto import trange, tqdm
import torch
import matplotlib.pyplot as plt
import numpy as np
import subprocess
from pathlib import Path
from datasets import Audio, load_dataset
import datasets
import shutil

# datasets.set_caching_enabled(False)

In [3]:
# Load pretrained model

pretrain_dataset = "mtat" # mtat, msd or jamendo
device = "cuda" if torch.cuda.is_available() else "cpu"
hcnn = model.HarmonicCNN().to(device)
state_dict = torch.load(f'sota-music-tagging-models/models/{pretrain_dataset}/hcnn/best_model.pth', map_location=device)
hcnn.load_state_dict(state_dict)
hcnn.eval();

In [4]:
# Load MusicCaps dataset

ds = load_musiccaps(
    './music_data',
    sampling_rate=16000,
    limit=None,
    num_proc=8,
    writer_batch_size=1000,
)

Using custom data configuration google--MusicCaps-7925612b943f961b
Found cached dataset csv (/home/dominik/.cache/huggingface/datasets/google___csv/google--MusicCaps-7925612b943f961b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Loading cached processed dataset at /home/dominik/.cache/huggingface/datasets/google___csv/google--MusicCaps-7925612b943f961b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-3309f9007425eecd.arrow


Loading cached processed dataset at /home/dominik/.cache/huggingface/datasets/google___csv/google--MusicCaps-7925612b943f961b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-70279f61db8a5fbb.arrow


Loading cached processed dataset at /home/dominik/.cache/huggingface/datasets/google___csv/google--MusicCaps-7925612b943f961b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-60f7e51bf7ab1aae.arrow


Loading cached processed dataset at /home/dominik/.cache/huggingface/datasets/google___csv/google--MusicCaps-7925612b943f961b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-54cb21bf4ee59f82.arrow


Loading cached processed dataset at /home/dominik/.cache/huggingface/datasets/google___csv/google--MusicCaps-7925612b943f961b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-6cb3cbe00008b97a.arrow


Loading cached processed dataset at /home/dominik/.cache/huggingface/datasets/google___csv/google--MusicCaps-7925612b943f961b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-1a5fb4f6dff5ee60.arrow


Loading cached processed dataset at /home/dominik/.cache/huggingface/datasets/google___csv/google--MusicCaps-7925612b943f961b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-fbf6f8ab350eeb18.arrow


Loading cached processed dataset at /home/dominik/.cache/huggingface/datasets/google___csv/google--MusicCaps-7925612b943f961b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-810290b7b9443a8c.arrow


In [19]:
activations = []
embedding = {}

# Define a hook function to store the activations in variable embedding
def hook(module, input, output):
    global activations
    activations.append(output.numpy(force=True)[0])

dense1 = hcnn.dense1
handle = dense1.register_forward_hook(hook)

for i in trange(len(ds)):
    try:
        audio = ds[i]['audio']['array']
    except FileNotFoundError as e:
        print(e)
        continue
        
    array_batch = torch.tensor(np.array([audio]*2)).to(device)

    # extract 256-embeddings of each chunk
    hcnn(array_batch[:, :80000])
    hcnn(array_batch[:, -80000:])
    
    assert len(activations) == 2
    embedding[ds[i]['ytid']] = np.concatenate(activations)
    activations.clear()

handle.remove()

  0%|          | 0/5521 [00:00<?, ?it/s]

[Errno 2] No such file or directory: 'music_data/0J_2K1Gvruk.wav'
[Errno 2] No such file or directory: 'music_data/374R7te0ra0.wav'
[Errno 2] No such file or directory: 'music_data/4X2aUZFZlzc.wav'
[Errno 2] No such file or directory: 'music_data/63rqIYPHvlc.wav'
[Errno 2] No such file or directory: 'music_data/7B1OAtD_VIA.wav'
[Errno 2] No such file or directory: 'music_data/Ah_aYOGnQ_I.wav'
[Errno 2] No such file or directory: 'music_data/Akg1n9IWSrw.wav'
[Errno 2] No such file or directory: 'music_data/B7iRvj8y9aU.wav'
[Errno 2] No such file or directory: 'music_data/BrCW3BA8TIQ.wav'
[Errno 2] No such file or directory: 'music_data/C7OIuhWSbjU.wav'
[Errno 2] No such file or directory: 'music_data/HAHn_zB47ig.wav'
[Errno 2] No such file or directory: 'music_data/NIcsJ8sEd0M.wav'
[Errno 2] No such file or directory: 'music_data/NXuB3ZEpM5U.wav'
[Errno 2] No such file or directory: 'music_data/RQ0-sjpAPKU.wav'
[Errno 2] No such file or directory: 'music_data/SLq-Co_szYo.wav'
[Errno 2] 

In [20]:
np.save('embeddings.npy', embedding)

In [33]:
import pandas as pd

df = pd.DataFrame()
df['ytid'] = np.array(added_ytids)
df['error'] = np.array([(i in errors) for i in added_ytids])
df['is_eval'] = False
df.to_csv('train.csv')

In [1]:
import numpy as np

In [3]:
embedding = np.load('embeddings.npy', allow_pickle=True)

In [9]:
embedding.item()['-0SdAVK79lg']

array([-1.0232261 ,  0.65255815, -0.90592617,  0.40343457,  0.7528761 ,
        0.44112068,  1.1816624 ,  0.1489468 ,  0.47368824, -0.84645396,
        1.2258904 , -0.75822836, -0.1097295 ,  0.793543  ,  0.4643134 ,
       -1.0545403 , -0.17713355, -0.4104601 ,  0.31560442, -1.1092676 ,
        1.487782  , -0.35267922,  0.12887435, -0.01217732, -0.31558278,
        0.3572585 , -1.1407242 , -0.61813253,  0.14208415, -0.33566245,
        0.18037766, -0.9543133 ,  0.50990534, -1.0415055 ,  0.17044885,
        0.11695306, -0.00920433, -0.64323974, -0.963087  , -0.4968316 ,
       -1.0901482 , -0.40098634,  1.0455302 , -0.40481687, -0.41257462,
       -0.95260847, -0.86906016,  0.0240692 ,  0.17121753, -0.14313461,
       -0.53836703,  0.2485201 ,  0.96019274, -0.44823575,  0.14670892,
       -0.94258   , -0.39383304,  0.4811812 ,  1.1378978 ,  0.6388795 ,
       -0.22472616, -0.9343545 , -0.5748392 ,  0.3611422 ,  0.19255471,
       -1.5191824 ,  1.56093   ,  0.02323203,  0.5410108 , -0.27

In [40]:
df.head()

,ytid,error,is_eval
0,-0SdAVK79lg,False,False
1,-1LrH01Ei1w,False,False
2,-4NLarMj4xU,False,False
3,-5f6hjZf9Yw,False,False
4,-5xOcMJpTUk,False,False


In [ ]:
with open('embeddings_eval.npy', 'wb') as f:
    np.save(f, embeddings)
with open('errors_eval.npy', 'wb') as f:
    np.save(f, np.array(errors))
with open('ytids_eval.npy', 'wb') as f:
    np.save(f, np.array(added_ytids))

NameError: ignored

In [ ]:
added_ytids

In [ ]:
errors